In [52]:
%load_ext autoreload
%aimport os, pandas, numpy, pickle
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import sys
import os
import pickle
import argparse
import pandas as pd
from sklearn.ensemble.forest import ExtraTreesRegressor

from fqi.et_tuning import run_tuning
from fqi.reward_function import *
from fqi.sars_creator import to_SARS
from fqi.utils import *

from trlib.policies.valuebased import EpsilonGreedy, Softmax
from trlib.policies.qfunction import ZeroQ
from trlib.algorithms.reinforcement.fqi_driver import FQIDriver, DoubleFQIDriver
from trlib.environments.trackEnv import TrackEnv
from trlib.utilities.ActionDispatcher import *

from fqi.fqi_evaluate import run_evaluation

In [54]:
simulations = pd.read_csv('./trajectory/dataset_offroad.csv',
                              dtype={'isReference': bool, 'is_partial':bool})

#simulations = simulations[(simulations['NLap'] == 1) | (simulations['NLap'] == 9)]

ref_tr = pd.read_csv('./trajectory/ref_traj.csv')
ref_tr.columns = ['time', 'Acceleration_x', 'Acceleration_y', 'speed_x', 'speed_y',
       'xCarWorld', 'yCarWorld', 'alpha_step']

In [55]:
file_name = 'model_file/penalty/penalty.pkl'
with open('./' + file_name, 'rb') as pen:
     penalty = pickle.load(pen)

In [42]:
def compute_offroad_penalty(X, trackPos):
        logp = penalty.kde.score_samples(X)
        #penalty.alpha * logp + penalty.scale_f
        mask = np.absolute(trackPos) > 1
        trackPos[~mask] = 0
        trackPos[mask] = np.absolute(trackPos[mask])*100
        return penalty.alpha * logp + penalty.scale_f + trackPos, trackPos
    
def compute_penalty(X, trackPos):
        logp = penalty.kde.score_samples(X)
        #penalty.alpha * logp + penalty.scale_f
        mask = np.absolute(trackPos) > 1
        trackPos[~mask] = 0
        trackPos[mask] = -50 #np.absolute(trackPos[mask])**5
        print((penalty.alpha * logp + penalty.scale_f).shape)
        print(trackPos.shape)
        return penalty.alpha * logp + penalty.scale_f + trackPos

In [41]:
(simulations['trackPos'].values).shape

(51702,)

In [40]:
rf = Temporal_projection(ref_tr, penalty=penalty, offroad = True, clip_range=(-np.inf, np.inf))

[autoreload of fqi.reward_function failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 384, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 323, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 278, in update_class
    if old_obj == new_obj:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
]


In [44]:
data = pd.read_csv('./trajectory/dataset_offroad.csv',
                              dtype={'isReference': bool, 'is_partial':bool})
NLap = 12
data = data[data['NLap'] == NLap]
episodes = {}
for e in np.unique(data['NLap']):
    mask = data['NLap'] == e
    lap_df = data[mask]
    n_samples = np.count_nonzero(mask)
    # create timestamp column
    timestamp = pd.DataFrame({'t': np.zeros([n_samples - 1])})
    # Create NLap column
    nlap = pd.DataFrame({'NLap': np.ones([n_samples - 1]) * e}, dtype=int)
    # Add state and action
    state_action = lap_df[state_cols+action_cols].iloc[:-1].reset_index(drop=True)
    # Add reward
    #reward = pd.DataFrame({'r': reward_function(lap_df)})
    r = rf(lap_df)
    #print(r)
    print('type:', type(r))
    print('shape', r.shape)

NameError: name 'offroad' is not defined

In [45]:
#off_p = compute_offroad_penalty(data[state_cols].values[1:], data['trackPos'].values[1:])
X = data[state_cols].values[1:]
trackPos = data['trackPos'].values[1:]
logp = penalty.kde.score_samples(X)
mask = np.absolute(trackPos) > 1
trackPos[~mask] = 0
trackPos[mask] = -50
print((penalty.alpha * logp + penalty.scale_f).shape)
print(trackPos.shape)
#penalty.alpha * logp + penalty.scale_f + trackPos

(849,)
(849,)


In [46]:
logp = penalty.kde.score_samples(X)
(penalty.alpha * logp + penalty.scale_f).shape

(849,)

In [42]:
pd.DataFrame(trackPos, columns=['trackPos']).describe()

,trackPos
count,849.000000
mean,44.769514
std,143.334689
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1007.229055
